# Part 1: Database and Jupyter Set Up

In [41]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [42]:
# Create an instance of MongoClient
# The default port used by MongoDB is 27017
mongo = MongoClient(port=27017)

In [43]:
# Make sure to import json file and create a database called uk_food and a collection called establsihments
# mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [44]:
# Examine available databases
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [45]:
# Assign the uk_food database to a variable name
uk_food = mongo['uk_food']

In [46]:
# Review the collections in the database
print(uk_food.list_collection_names())

['establishments']


In [47]:
# assign collection to a variable
establishments = uk_food['establishments']

In [48]:
# Review a document in the collection 
pprint(establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63cdeb79a0392a1bef670dad'),
 'geocode': {'latitude': 51.083812, 'longitude': Decimal128('1.195625')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dat

# Part 2: Update the Database

## (1) Insert the halal restaurant to the uk_food database

In [49]:
# Build a dictionary for the new restuarant to add to the database
halal_rest = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [50]:
# Insert the new restaurant to the collection
establishments.insert_one(halal_rest)

In [51]:
# Check that the new restaurant was added; print "True" if the query for the new restaurant turns up empty
print(establishments.find({"BusinessName":"Penang Flavours"}) == None)

False


## (2) Find the BusinessTypeID for restaurants

In [52]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType":"Restaurant/Cafe/Canteen"}
fields = {'BusinessTypeID':1, 'BusinessType':1}

results_business_type = list(establishments.find(query, fields))

In [53]:
# Print the results
pprint(results_business_type[0])

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('63cdeb79a0392a1bef670dad')}


## (3) Update the BusinessTypeID for the new halal restaurant

In [54]:
# Update Penang Flavors with the correct business type 
uk_food.establishments.update_one(
    {"BusinessName":"Penang Flavours"},
    {'$set': 
        {'BusinessTypeID': 1}}
)

In [55]:
# Confirm that the new restaurant was updated
pprint(establishments.find({"BusinessName":"Penang Flavours"})[0])

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63cdf125cfceb4e3db666072'),
 'geocode': {'latitude': 51.490142, 'longitude': Decimal128('0.08384000')},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


## (4) Remove any establishments with Dover Local Authority

### Find how many documents have LocalAuthorityName as "Dover"

In [56]:
query = {'LocalAuthorityName': 'Dover'}
len(list(establishments.find(query)))

0

### Delete all documents with the 'LocalAuthorityName' as 'Dover'

In [57]:
uk_food.establishments.delete_many(query)

### Check if any other 'Dover' documents still exists

In [58]:
# Check how many documents exists still with 'Dover' as its 'LocalAuthorityName
len(list(establishments.find(query)))

0

### Check that there are still other documents

In [59]:
# Check how many documents remain within the establsihments collection
len(list(establishments.find({})))

38787

## (5) Convert the latitude and longitude to decimal

### Change the data type from String to Decimal for longitude

In [60]:
# Use Update many to change the latitude to decimal
establishments.update_many({}, [ {'$set':{'geocode.latitude': {'$toDouble': '$geocode.latitude'}}}])

In [61]:
# Check to make sure it worked! 
pprint(establishments.find_one({}, {'geocode'}))

{'_id': ObjectId('63cdeb79a0392a1bef670dad'),
 'geocode': {'latitude': 51.083812, 'longitude': Decimal128('1.195625')}}


### Change the data type from String to Decimal for latitude

In [62]:
# Change the data type from String to Decimal for longitude
establishments.update_many({}, [ {'$set':{'geocode.longitude': {'$toDouble': '$geocode.longitude'}}}])

In [63]:
# Check to make sure it worked! 
pprint(establishments.find_one({}, {'geocode'}))

{'_id': ObjectId('63cdeb79a0392a1bef670dad'),
 'geocode': {'latitude': 51.083812, 'longitude': Decimal128('1.195625')}}


In [64]:
mongo.close()